Coordinator Logic (Rule-Based)

In [7]:
from pathlib import Path
import os

PROJECT_ROOT = Path(r"C:\Users\Karun\OneDrive\Downloads\Clause AI\Notebooks\artifacts")
os.chdir(PROJECT_ROOT)

print("Working directory set to:", Path.cwd())


Working directory set to: C:\Users\Karun\OneDrive\Downloads\Clause AI\Notebooks\artifacts


1: Load Existing Agent Outputs

In [8]:
import json

with open("legal_agent_output.json") as f:
    legal_output = json.load(f)

with open("compliance_agent_output.json") as f:
    compliance_output = json.load(f)

with open("finance_agent_output.json") as f:
    finance_output = json.load(f)

with open("operations_agent_output.json") as f:
    operations_output = json.load(f)


2: Define Routing Rules (add indemnity)
🔑 Question: Which agent handles indemnity?

Answer: 👉 Legal Agent

Why?

Indemnity = legal liability, damages, risk allocation

Falls under contract law

In [ ]:
ROUTING_RULES = {
    "legal": ["termination", "governing law", "jurisdiction", "indemnity"],
    "compliance": ["gdpr", "audit", "regulatory", "data protection"],
    "finance": ["payment", "fee", "penalty", "invoice"],
    "operations": ["deliverable", "timeline", "sla", "milestone"]
}


3: Define Routing Function

This function decides which agents to activate.

✔️ Supports:

Single-agent routing

Multi-agent routing

In [10]:
def route_query(query):
    query = query.lower()
    selected_agents = []

    for agent, keywords in ROUTING_RULES.items():
        for keyword in keywords:
            if keyword in query:
                selected_agents.append(agent)
                break  # avoid duplicate adds

    return selected_agents


4: Test Routing Logic

Test 1: Indemnity

In [11]:
query = "Explain indemnity obligations in this contract"
print(route_query(query))


['legal']


Test 2: Multi-agent query

In [12]:
query = "Are there any indemnity clauses and payment penalties?"
print(route_query(query))


['legal', 'finance']


Test 3: Compliance + Ops

In [13]:
query = "Does the SLA comply with regulatory audit requirements?"
print(route_query(query))


['compliance', 'operations']


5: Coordinator Execution Logic

Now we combine routing + outputs.

In [14]:
def coordinator_execute(query):
    agents = route_query(query)
    results = {}

    for agent in agents:
        if agent == "legal":
            results["legal"] = legal_output

        elif agent == "compliance":
            results["compliance"] = compliance_output

        elif agent == "finance":
            results["finance"] = finance_output

        elif agent == "operations":
            results["operations"] = operations_output

    return results


6: Run Coordinator

In [15]:
query = "Check indemnity clauses and payment penalties"
final_output = coordinator_execute(query)


7: View Coordinator Output

In [ ]:
import pprint
pprint.pprint(final_output)


{'finance': {'agent_name': 'Finance Agent',
             'input_summary': {'combined_text_length': 8365,
                               'num_context_chunks': 12,
                               'source_queries': ['What are the payment terms '
                                                  'and conditions?',
                                                  'What are the fees, '
                                                  'invoices, and billing '
                                                  'requirements?',
                                                  'What are the penalties and '
                                                  'late fees for non-payment?',
                                                  'What is the financial '
                                                  'liability and '
                                                  'indemnification?']},
             'model': 'google/gemma-2b-it',
             'output': {'clause_type': 'Finance Analysis'

Save Coordinator Output to a File

In [17]:
def save_coordinator_output(query, output):
    filename = "coordinator_output.json"
    with open(filename, "w") as f:
        json.dump({
            "query": query,
            "results": output
        }, f, indent=4)

save_coordinator_output(query, final_output)


LangGraph Basics

1: Install & Import LangGraph

In [18]:
pip install langgraph




   ------------- -------------------------- 2/6 [langgraph-sdk]
   -------------------- ------------------- 3/6 [langgraph-checkpoint]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   ---------------------------------------- 6/6 [langgraph]



In [19]:
from langgraph.graph import StateGraph, END
from typing import TypedDict


2: Define Shared Graph State

All nodes read/write to this state.

In [32]:
from typing import TypedDict
class GraphState(TypedDict):
    query: str
    legal: dict
    compliance: dict
    query: str
    compliance: dict
    legal: dict
    finance: dict
    operations: dict



3: Define Agent Nodes (with print logs ✅)

You already have agent outputs, so we’ll simulate execution using those outputs.

Compliance Node

In [33]:
def compliance_node(state: GraphState):
    print("🟡 Entering Compliance Node")

    state["compliance"] = {
        "issues": ["GDPR compliance risk"],
        "risk_level": "Medium"
    }

    print("🟡 Exiting Compliance Node")
    return state


Legal Node

In [34]:
def legal_node(state: GraphState):
    print("🔵 Entering Legal Node")

    state["legal"] = {
        "clauses": ["Indemnity clause found"],
        "risk_level": "High"
    }

    print("🔵 Exiting Legal Node")
    return state


Finance Node

In [35]:
def finance_node(state: GraphState):
    print("🟢 Entering Finance Node")

    state["finance"] = {
        "payment_terms": ["Net 30 days"],
        "penalties": ["Late payment penalty applies"],
        "financial_risk": "Medium"
    }

    print("🟢 Exiting Finance Node")
    return state


Operations Node

In [36]:
def operations_node(state: GraphState):
    print("🟣 Entering Operations Node")

    state["operations"] = {
        "deliverables": ["Monthly reports"],
        "sla": "99.9% uptime",
        "operational_risk": "Low"
    }

    print("🟣 Exiting Operations Node")
    return state


4: Build Graph Skeleton

In [37]:
graph = StateGraph(GraphState)


5: Add Nodes to Graph

In [38]:
graph.add_node("compliance", compliance_node)
graph.add_node("legal", legal_node)
graph.add_node("finance", finance_node)
graph.add_node("operations", operations_node)



6: Define Edges

In [39]:
graph.set_entry_point("compliance")

graph.add_edge("compliance", "legal")
graph.add_edge("legal", "finance")
graph.add_edge("finance", "operations")
graph.add_edge("operations", END)


7: Compile Graph

In [40]:
compiled_graph = graph.compile()


8: Execute Graph

In [41]:
initial_state = {
    "query": "Check indemnity, GDPR, payment terms and SLA",
    "compliance": {},
    "legal": {},
    "finance": {},
    "operations": {}
}

final_state = compiled_graph.invoke(initial_state)


🟡 Entering Compliance Node
🟡 Exiting Compliance Node
🔵 Entering Legal Node
🔵 Exiting Legal Node
🟢 Entering Finance Node
🟢 Exiting Finance Node
🟣 Entering Operations Node
🟣 Exiting Operations Node


9: Inspect Output

In [42]:
print("\n✅ FINAL GRAPH OUTPUT")
print(final_state)



✅ FINAL GRAPH OUTPUT
{'query': 'Check indemnity, GDPR, payment terms and SLA', 'legal': {'clauses': ['Indemnity clause found'], 'risk_level': 'High'}, 'compliance': {'issues': ['GDPR compliance risk'], 'risk_level': 'Medium'}, 'finance': {'payment_terms': ['Net 30 days'], 'penalties': ['Late payment penalty applies'], 'financial_risk': 'Medium'}, 'operations': {'deliverables': ['Monthly reports'], 'sla': '99.9% uptime', 'operational_risk': 'Low'}}


Save LangGraph Output to JSON

In [44]:
import json

with open("langgraph_output.json", "w") as f:
    json.dump(final_state, f, indent=4)

print("✅ Full LangGraph output saved")


✅ Full LangGraph output saved


Multi-Agent Graph (Nodes & Edges)

1: Define Expanded Graph State

In [45]:
from typing import TypedDict

class GraphState(TypedDict):
    query: str
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


2️: Define Agent Nodes (with logs)

In [46]:
#LEGAL_AGENT_NODE
def legal_node(state: GraphState):
    print("🔵 Legal Agent START")

    state["legal"] = {
        "termination": "Termination with 30 days notice",
        "indemnity": "Indemnity clause present",
        "risk": "High"
    }

    print("🔵 Legal Agent END")
    return state


In [47]:
#COMPILANCE_AGENT_NODE
def compliance_node(state: GraphState):
    print("🟡 Compliance Agent START")

    state["compliance"] = {
        "gdpr": "Data processing clause present",
        "audit": "Annual audit required",
        "risk": "Medium"
    }

    print("🟡 Compliance Agent END")
    return state


In [48]:
#FINANCE_AGENT_NODE
def finance_node(state: GraphState):
    print("🟢 Finance Agent START")

    state["finance"] = {
        "payment_terms": "Net 30",
        "penalty": "2% late fee",
        "risk": "Medium"
    }

    print("🟢 Finance Agent END")
    return state


In [49]:
#OPERATIONS_AGENT_NODE
def operations_node(state: GraphState):
    print("🟣 Operations Agent START")

    state["operations"] = {
        "sla": "99.9% uptime",
        "deliverables": "Monthly reports",
        "risk": "Low"
    }

    print("🟣 Operations Agent END")
    return state


3: Create LangGraph

In [50]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)


4: Add Nodes to Graph

In [51]:
graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)


5: Define Execution Flow (Sequential)

In [52]:
#Defaultflow: Legal → Compliance → Finance → Operations


graph.set_entry_point("legal_agent")

graph.add_edge("legal_agent", "compliance_agent")
graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "operations_agent")
graph.add_edge("operations_agent", END)


6: Compile Graph

In [53]:
app = graph.compile()


7: Execute Multi-Agent Graph

In [54]:
input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(input_state)
print(result.keys())


🔵 Legal Agent START
🔵 Legal Agent END
🟡 Compliance Agent START
🟡 Compliance Agent END
🟢 Finance Agent START
🟢 Finance Agent END
🟣 Operations Agent START
🟣 Operations Agent END
dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])


8: Save Multi-Agent Graph to JSON

In [62]:
import json

with open("multi_agent_langgraph_output.json", "w") as f:
    json.dump(result, f, indent=4)

print("✅ Multi-agent LangGraph output saved successfully")


✅ Multi-agent LangGraph output saved successfully


✅ TASK 1: Change Execution Order

New order:
Compliance → Legal → Operations → Finance

In [65]:
#change edges only

graph = StateGraph(GraphState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")

graph.add_edge("compliance_agent", "legal_agent")
graph.add_edge("legal_agent", "operations_agent")
graph.add_edge("operations_agent", "finance_agent")
graph.add_edge("finance_agent", END)

app = graph.compile()
print(app)


In [66]:
result = app.invoke(input_state)
print(result)

🟡 Compliance Agent START
🟡 Compliance Agent END
🔵 Legal Agent START
🔵 Legal Agent END
🟣 Operations Agent START
🟣 Operations Agent END
🟢 Finance Agent START
🟢 Finance Agent END
{'query': 'Review termination, GDPR compliance, payment terms, and SLAs', 'legal': {'termination': 'Termination with 30 days notice', 'indemnity': 'Indemnity clause present', 'risk': 'High'}, 'compliance': {'gdpr': 'Data processing clause present', 'audit': 'Annual audit required', 'risk': 'Medium'}, 'finance': {'payment_terms': 'Net 30', 'penalty': '2% late fee', 'risk': 'Medium'}, 'operations': {'sla': '99.9% uptime', 'deliverables': 'Monthly reports', 'risk': 'Low'}}


Observe logs:
🟡 Compliance → 🔵 Legal → 🟣 Operations → 🟢 Finance


✔ Order changed without touching agent code — LangGraph power.

✅ TASK 2: Remove One Agent (Finance) and Re-Run


Remove:
finance_agent

In [67]:
#Modified_Graph

graph = StateGraph(GraphState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("legal_agent")

graph.add_edge("legal_agent", "compliance_agent")
graph.add_edge("compliance_agent", "operations_agent")
graph.add_edge("operations_agent", END)

app = graph.compile()


In [68]:
result = app.invoke(input_state)
print(result)


🔵 Legal Agent START
🔵 Legal Agent END
🟡 Compliance Agent START
🟡 Compliance Agent END
🟣 Operations Agent START
🟣 Operations Agent END
{'query': 'Review termination, GDPR compliance, payment terms, and SLAs', 'legal': {'termination': 'Termination with 30 days notice', 'indemnity': 'Indemnity clause present', 'risk': 'High'}, 'compliance': {'gdpr': 'Data processing clause present', 'audit': 'Annual audit required', 'risk': 'Medium'}, 'finance': {}, 'operations': {'sla': '99.9% uptime', 'deliverables': 'Monthly reports', 'risk': 'Low'}}


Observe:

finance remains {}

Other agents still populate their state

Conditional Routing in LangGraph

1: Define Routing Function

This function decides which agent should run first.

In [69]:
def route_query(state):
    query = state["query"].lower()

    if any(word in query for word in ["termination", "indemnity", "jurisdiction"]):
        return "legal_agent"

    if any(word in query for word in ["gdpr", "audit", "compliance", "data protection"]):
        return "compliance_agent"

    if any(word in query for word in ["payment", "penalty", "invoice", "fee"]):
        return "finance_agent"

    if any(word in query for word in ["sla", "timeline", "deliverable", "milestone"]):
        return "operations_agent"

    return END


2: Rebuild Graph with Conditional Entry

We now use conditional entry, not a fixed one.

In [70]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)


STEP 3️⃣ Add Agent Nodes

(Same agents you already wrote)

In [71]:
graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)


4: Conditional Entry Point

This is the key LangGraph feature.

In [ ]:
graph.set_conditional_entry_point(
    route_query,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        END: END
    }
)

"""📌 Meaning:

LangGraph calls route_query(state)

Whatever string it returns → that node runs
"""


5: Agent → END Edges

Each agent ends execution after it runs.

In [73]:
graph.add_edge("legal_agent", END)
graph.add_edge("compliance_agent", END)
graph.add_edge("finance_agent", END)
graph.add_edge("operations_agent", END)


6: Compile Graph

In [74]:
app = graph.compile()


7: TEST CASES

In [ ]:
#Test Case 1: Legal Query
state = {
    "query": "Review termination clause",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)
print(result.keys())

'''
Observe:
Only Legal Agent logs appear
Other states remain {}
'''


🔵 Legal Agent START
🔵 Legal Agent END
dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])


In [ ]:
#Test Case 2: Finance Query
state = {
    "query": "Check late payment penalties",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)
print(result.keys())


'''Observe logs:
🟢 Finance Agent START
🟢 Finance Agent END
✔ Only Finance executed
'''

🟢 Finance Agent START
🟢 Finance Agent END
dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])


In [ ]:
#Test Case 3: Multiple Intent - limitation
state = {
    "query": "Check GDPR compliance and payment terms",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)
print(result)


🟡 Compliance Agent START
🟡 Compliance Agent END
{'query': 'Check GDPR compliance and payment terms', 'legal': {}, 'compliance': {'gdpr': 'Data processing clause present', 'audit': 'Annual audit required', 'risk': 'Medium'}, 'finance': {}, 'operations': {}}


✅ Test Multiple Queries

You can batch test like this:

In [79]:
queries = [
    "Review indemnity clause",
    "Check GDPR audit requirement",
    "Late payment penalty",
    "Verify SLA uptime"
]

for q in queries:
    print("\nQuery:", q)
    state = {
        "query": q,
        "legal": {},
        "compliance": {},
        "finance": {},
        "operations": {}
    }
    app.invoke(state)



Query: Review indemnity clause
🔵 Legal Agent START
🔵 Legal Agent END

Query: Check GDPR audit requirement
🟡 Compliance Agent START
🟡 Compliance Agent END

Query: Late payment penalty
🟢 Finance Agent START
🟢 Finance Agent END

Query: Verify SLA uptime
🟣 Operations Agent START
🟣 Operations Agent END


✅ Observe Which Agent Is Selected

✔ Logs show it
✔ State shows it
✔ Other agents are untouched

This proves true conditional execution.

Save Conditional Routing Output to JSON

Right now:

LangGraph runs only ONE relevant agent
Other agents stay {}
We want to persist this intelligently

In [ ]:
#RIdentify Which Agent Actually Ran
def get_executed_agents(result):
    return [
        agent for agent in ["legal", "compliance", "finance", "operations"]
        if result.get(agent)
    ]

executed_agents = get_executed_agents(result)
print(executed_agents)


#Build Conditional Output Payload
#Now we store: query, selected agent(s), only meaningful outputs

from datetime import datetime
output_payload = {
    "query": result["query"],
    "timestamp": datetime.now().isoformat(),
    "executed_agents": executed_agents,
    "results": {
        agent: result[agent] for agent in executed_agents
    }
}


#Save file to JSON
import json
with open("conditional_langgraph_output.json", "w") as f:
    json.dump(output_payload, f, indent=4)

print("✅ Conditional LangGraph output saved to JSON")


['compliance']
✅ Conditional LangGraph output saved to JSON


Conversation Memory & State Persistence (Agent Memory)

1: Update Graph State (Add Memory + Validation Notes)

We slightly extend your state to support validation.

In [85]:
from typing import TypedDict, List

class GraphState(TypedDict):
    query: str
    memory: List[dict]
    validation_notes: List[str]
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


2: Initialize Memory in Input Stat

In [86]:
input_state = {
    "query": "Check GDPR compliance, payment penalties, indemnity and SLA enforceability",
    "memory": [],
    "validation_notes": [],
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}


3: Compliance Agent (Writes to Memory)

✔ Writes findings
✔ Appends to memory
✔ Prints memory (your task ✅)

In [87]:
def compliance_node(state: GraphState):
    print("🟡 Compliance Agent START")

    output = {
        "extracted_clauses": ["GDPR data processing clause", "Audit rights clause"],
        "risk": "Medium"
    }

    state["compliance"] = output

    state["memory"].append({
        "agent": "compliance",
        "findings": output["extracted_clauses"]
    })

    print("🧠 Memory after Compliance:", state["memory"])
    print("🟡 Compliance Agent END")

    return state


4: Finance Agent (Reads Compliance Memory + Writes Memory)

✔ Reads compliance memory
✔ Adds validation note
✔ Appends own findings

In [94]:
def finance_node(state: GraphState):
    print("🟢 Finance Agent START")

    compliance_findings = [
        m for m in state["memory"] if m["agent"] == "compliance"
    ]

    output = {
        "extracted_clauses": ["Late payment penalty: 2%", "Net 30 payment term"],
        "risk": "Medium"
    }

    state["finance"] = output

    if compliance_findings:
        state["validation_notes"].append(
            "Finance reviewed compliance clauses for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": output["extracted_clauses"]
    })

    print("🧠 Memory after Finance:", state["memory"])
    print("🟢 Finance Agent END")

    return state
    


5: Legal Agent (Final Validation + Memory Logging)

✔ Legal agent memory logging added
✔ Memory printed
✔ Validation added

In [95]:
def legal_node(state: GraphState):
    print("🔵 Legal Agent START")

    finance_findings = [
        m for m in state["memory"] if m["agent"] == "finance"
    ]

    output = {
        "extracted_clauses": ["Indemnity clause", "Termination clause"],
        "risk": "High"
    }

    state["legal"] = output

    if finance_findings:
        state["validation_notes"].append(
            "Legal validated penalty clauses against indemnity exposure."
        )

    state["memory"].append({
        "agent": "legal",
        "findings": output["extracted_clauses"]
    })

    print("🧠 Memory after Legal:", state["memory"])
    print("🔵 Legal Agent END")

    return state


Agent-to-Agent Communication (Operations reads Legal)

6: Operations Agent (Reads Legal Output)

✔ Reads Legal output via memory
✔ Adds SLA enforceability note

In [96]:
def operations_node(state: GraphState):
    print("🟣 Operations Agent START")

    legal_findings = [
        m for m in state["memory"] if m["agent"] == "legal"
    ]

    output = {
        "extracted_clauses": ["SLA uptime: 99.9%", "Service credits clause"],
        "risk": "Low"
    }

    state["operations"] = output

    if legal_findings:
        state["validation_notes"].append(
            "Operations validated SLA enforceability against legal clauses."
        )

    state["memory"].append({
        "agent": "operations",
        "findings": output["extracted_clauses"]
    })

    print("🧠 Memory after Operations:", state["memory"])
    print("🟣 Operations Agent END")

    return state


7: Build Collaborative Graph

In [97]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("legal_agent", legal_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")

graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "legal_agent")
graph.add_edge("legal_agent", "operations_agent")
graph.add_edge("operations_agent", END)


8: Compile Graph

In [98]:
app = graph.compile()

9: Execute Collaborative Flow

In [99]:
result = app.invoke(input_state)

🟡 Compliance Agent START
🧠 Memory after Compliance: [{'agent': 'compliance', 'findings': ['GDPR data processing clause', 'Audit rights clause']}]
🟡 Compliance Agent END
🟢 Finance Agent START
🧠 Memory after Finance: [{'agent': 'compliance', 'findings': ['GDPR data processing clause', 'Audit rights clause']}, {'agent': 'finance', 'findings': ['Late payment penalty: 2%', 'Net 30 payment term']}]
🟢 Finance Agent END
🔵 Legal Agent START
🧠 Memory after Legal: [{'agent': 'compliance', 'findings': ['GDPR data processing clause', 'Audit rights clause']}, {'agent': 'finance', 'findings': ['Late payment penalty: 2%', 'Net 30 payment term']}, {'agent': 'legal', 'findings': ['Indemnity clause', 'Termination clause']}]
🔵 Legal Agent END
🟣 Operations Agent START
🧠 Memory after Operations: [{'agent': 'compliance', 'findings': ['GDPR data processing clause', 'Audit rights clause']}, {'agent': 'finance', 'findings': ['Late payment penalty: 2%', 'Net 30 payment term']}, {'agent': 'legal', 'findings': ['I

10: Inspect Final Memory & Validation Notes


Expected Memory Accumulation Order

Compliance → Finance → Legal → Operations
✔ Confirms multi-step reasoning
✔ Confirms agent collaboration

In [100]:
print("\n🧠 FINAL MEMORY")
for item in result["memory"]:
    print(item)

print("\n✅ VALIDATION NOTES")
for note in result["validation_notes"]:
    print("-", note)



🧠 FINAL MEMORY
{'agent': 'compliance', 'findings': ['GDPR data processing clause', 'Audit rights clause']}
{'agent': 'finance', 'findings': ['Late payment penalty: 2%', 'Net 30 payment term']}
{'agent': 'legal', 'findings': ['Indemnity clause', 'Termination clause']}
{'agent': 'operations', 'findings': ['SLA uptime: 99.9%', 'Service credits clause']}

✅ VALIDATION NOTES
- Finance reviewed compliance clauses for penalty conflicts.
- Legal validated penalty clauses against indemnity exposure.
- Operations validated SLA enforceability against legal clauses.


Save Conversation Memory & Agent State to JSON

In [101]:
from datetime import datetime

output_payload = {
    "query": result["query"],
    "timestamp": datetime.now().isoformat(),
    "memory": result["memory"],
    "validation_notes": result["validation_notes"],
    "agent_outputs": {
        "compliance": result["compliance"],
        "finance": result["finance"],
        "legal": result["legal"],
        "operations": result["operations"]
    }
}


In [102]:
import json

with open("agent_memory_output.json", "w") as f:
    json.dump(output_payload, f, indent=4)

print("✅ Agent memory and validation saved to agent_memory_output.json")


✅ Agent memory and validation saved to agent_memory_output.json


PIPELINES

Compliance Pipeline


The compliance pipeline retrieves relevant contract context, extracts compliance clauses, validates them, and produces a structured risk summary.

1: Define Compliance Query Template

In [5]:
from pathlib import Path
import os

PROJECT_ROOT = Path(r"C:\Users\Karun\OneDrive\Downloads\Clause AI\Notebooks\artifacts")
os.chdir(PROJECT_ROOT)

print("Working directory set to:", Path.cwd())


Working directory set to: C:\Users\Karun\OneDrive\Downloads\Clause AI\Notebooks\artifacts


In [6]:
COMPLIANCE_QUERY = """
Identify clauses related to:
- Regulatory compliance
- Data protection
- Audits and reporting
"""

#This is the instruction passed to the Compliance Agent.

2: Retrieve Compliance Context (RAG)

Assume you already have a vector store / chunks.

In [7]:
#Initial Retrieval Keywords
retrieval_keywords = [
    "compliance",
    "regulatory",
    "data protection",
    "audit"
]

#Example Retriever Function
def retrieve_compliance_context(chunks, keywords):
    retrieved = []
    for chunk in chunks:
        if any(k.lower() in chunk.lower() for k in keywords):
            retrieved.append(chunk)
    return retrieved


3: Combine Retrieved Chunks


This is what the agent actually reads.

In [8]:
import json
with open("rag_results.json", "r") as f:
    rag_results = json.load(f)
type(rag_results)  # list
contract_chunks = [item["text"] for item in rag_results]

In [9]:
retrieved_chunks = retrieve_compliance_context(
    contract_chunks,
    retrieval_keywords
)

combined_context = "\n\n".join(retrieved_chunks)


4: Run Compliance Agent

In [10]:
def run_compliance_agent(context):
    return {
        "extracted_clauses": [
            "The vendor shall comply with all applicable data protection laws.",
            "Annual audits may be conducted by the client."
        ],
        "risk_level": "Medium",
        "confidence": 0.78
    }


In [11]:
compliance_output = run_compliance_agent(combined_context)


5: Validate Compliance Output

In [12]:
def validate_compliance(output):
    issues = []

    if "data protection" not in " ".join(output["extracted_clauses"]).lower():
        issues.append("Missing explicit data protection clause")

    return {
        "is_valid": len(issues) == 0,
        "issues": issues
    }


In [13]:
validation_result = validate_compliance(compliance_output)


6: Compliance Risk Summary

In [14]:
compliance_summary = {
    "risk_level": compliance_output["risk_level"],
    "confidence": compliance_output["confidence"],
    "validated": validation_result["is_valid"]
}


7: Package Pipeline Output ✅

In [15]:
final_compliance_pipeline_output = {
    "query_template": COMPLIANCE_QUERY.strip(),
    "retrieved_chunks_count": len(retrieved_chunks),
    "extracted_clauses": compliance_output["extracted_clauses"],
    "validation": validation_result,
    "summary": compliance_summary
}
print(final_compliance_pipeline_output)

{'query_template': 'Identify clauses related to:\n- Regulatory compliance\n- Data protection\n- Audits and reporting', 'retrieved_chunks_count': 0, 'extracted_clauses': ['The vendor shall comply with all applicable data protection laws.', 'Annual audits may be conducted by the client.'], 'validation': {'is_valid': True, 'issues': []}, 'summary': {'risk_level': 'Medium', 'confidence': 0.78, 'validated': True}}


8: Save Compliance Pipeline Output to JSON

In [16]:
import json
from datetime import datetime
final_compliance_pipeline_output["timestamp"] = datetime.now().isoformat()


In [153]:
with open("compliance_pipeline_output.json", "w") as f:
    json.dump(final_compliance_pipeline_output, f, indent=4)

print("✅ Compliance pipeline output saved to compliance_pipeline_output.json")


✅ Compliance pipeline output saved to compliance_pipeline_output.json


✅ Task 1: Change Retrieval Keywords
🔄 New Keywords (Stricter)

In [17]:
retrieval_keywords = [
    "gdpr",
    "data processing",
    "regulatory authority",
    "audit rights"
]


In [18]:
retrieved_chunks = retrieve_compliance_context(
    contract_chunks,
    retrieval_keywords
)


✅ Task 2: Observe Clause Differences


Before (Generic keywords):
- General compliance obligations
- Audit may be conducted

After (Specific keywords):
- GDPR data processing obligations
- Regulatory authority access rights

Observation:
More specific keywords → fewer but more precise clauses.

✅ Task 3: Compare Confidence Scores

Simulate confidence logic:

In [19]:
def compute_confidence(num_clauses):
    if num_clauses >= 3:
        return 0.85
    elif num_clauses == 2:
        return 0.75
    else:
        return 0.6


In [20]:
confidence_old = compute_confidence(2)  # generic retrieval
confidence_new = compute_confidence(3)  # specific retrieval
print(confidence_new)
print(confidence_old)

'''
✔ Better retrieval → higher confidence
✔ Cleaner compliance output
'''

0.85
0.75


'\n✔ Better retrieval → higher confidence\n✔ Cleaner compliance output\n'

Finance Pipeline 


The finance pipeline retrieves payment-related clauses, extracts penalties/interest terms, validates them, and produces a financial risk summary.

1: Define Finance Query Template

This guides the Finance Agent.

In [21]:
FINANCE_QUERY = """
Identify clauses related to:
- Payment terms
- Late payment penalties
- Interest on overdue amounts
- Invoicing conditions
"""


2: Retrieve Finance Context (RAG)

In [22]:
#Initial Finance Retrieval Keywords
finance_keywords = [
    "payment",
    "penalty",
    "invoice"
]

#Retriever Function
def retrieve_finance_context(chunks, keywords):
    retrieved = []
    for chunk in chunks:
        if any(k.lower() in chunk.lower() for k in keywords):
            retrieved.append(chunk)
    return retrieved



3: Combine Retrieved Chunks

In [23]:
retrieved_finance_chunks = retrieve_finance_context(
    contract_chunks,
    finance_keywords
)

combined_finance_context = "\n\n".join(retrieved_finance_chunks)

print(combined_finance_context)


In [24]:
retrieved_finance_chunks = retrieve_finance_context(
    contract_chunks,
    finance_keywords
)

combined_finance_context = "\n\n".join(retrieved_finance_chunks)

print(combined_finance_context)


4: Run Finance Agent

In [25]:
def run_finance_agent(context):
    return {
        "extracted_clauses": [
            "Payment terms are Net 30 days from invoice date.",
            "Late payments may incur a penalty of 2% per month."
        ],
        "risk_level": "Medium",
        "confidence": 0.72
    }


In [26]:
finance_output = run_finance_agent(combined_finance_context)


5: Validate Finance Output

In [27]:
def validate_finance_output(output):
    issues = []

    clauses_text = " ".join(output["extracted_clauses"]).lower()

    if "payment" not in clauses_text:
        issues.append("Missing explicit payment terms")

    if "penalty" not in clauses_text and "interest" not in clauses_text:
        issues.append("No late payment consequence defined")

    return {
        "is_valid": len(issues) == 0,
        "issues": issues
    }


In [29]:
finance_validation = validate_finance_output(finance_output)


6: Finance Risk Summary

In [30]:
finance_summary = {
    "risk_level": finance_output["risk_level"],
    "confidence": finance_output["confidence"],
    "validated": finance_validation["is_valid"]
}


7: Package Finance Pipeline Output

In [31]:
final_finance_pipeline_output = {
    "query_template": FINANCE_QUERY.strip(),
    "retrieved_chunks_count": len(retrieved_finance_chunks),
    "extracted_clauses": finance_output["extracted_clauses"],
    "validation": finance_validation,
    "summary": finance_summary
}


8: Save to JSON

In [32]:
import json
from datetime import datetime
final_finance_pipeline_output["timestamp"] = datetime.now().isoformat()


with open("final_finance_pipeline_outputt.json", "w") as f:
    json.dump(final_finance_pipeline_output, f, indent=4)

print("✅ Finance pipeline output saved to compliance_pipeline_output.json")


✅ Finance pipeline output saved to compliance_pipeline_output.json


✅ Task 1: Add Keyword "interest"

Update finance keywords 👇

In [33]:
finance_keywords = [
    "payment",
    "penalty",
    "invoice",
    "interest"   # ✅ added
]


✅ Task 2: Re-Run Pipeline

In [34]:
retrieved_finance_chunks = retrieve_finance_context(
    contract_chunks,
    finance_keywords
)

combined_finance_context = "\n\n".join(retrieved_finance_chunks)
print(combined_finance_context)


ance
         interests of the Party  terminating its  participation in proportion to
         their interests assigned  immediately  preceding such effective date of
         termination,  except for the continuing  rights and  obligations of the
         terminating Party as specified in Subparagraphs  25.4, 25.5 and of this
         Agreement.  No credit  for  capital  costs will be made to a Party that
         terminates its participation in accordance with this Subparagraph 25.1.
         


(Re-run Finance Agent with Interest)

In [35]:
def run_finance_agent(context):
    return {
        "extracted_clauses": [
            "Payment terms are Net 30 days from invoice date.",
            "Late payments may incur a penalty of 2% per month.",
            "Interest of 18% per annum applies to overdue amounts."
        ],
        "risk_level": "High",   # ⬆️ increased
        "confidence": 0.85      # ⬆️ increased
    }


In [36]:
finance_output = run_finance_agent(combined_finance_context)
finance_validation = validate_finance_output(finance_output)
print(finance_output)
print(finance_validation)

{'extracted_clauses': ['Payment terms are Net 30 days from invoice date.', 'Late payments may incur a penalty of 2% per month.', 'Interest of 18% per annum applies to overdue amounts.'], 'risk_level': 'High', 'confidence': 0.85}
{'is_valid': True, 'issues': []}


✅ Task 3: Observe Risk Level Changes

🔍 Comparison
Scenario	            Clauses Found	              Risk Level	Confidence
Without “interest”  	Payment + Penalty	            Medium	       0.72
With “interest”	        Payment + Penalty + Interest	 High	       0.85

📌 Why risk increased?
Interest clauses amplify financial exposure
Higher compounding liability → higher risk

Legal Pipeline 

The legal pipeline retrieves legally relevant contract clauses, extracts structured legal obligations, validates them, and produces a legal risk summary.

1: Define Legal Query Template

This tells the Legal Agent what to look for.

In [37]:
LEGAL_QUERY = """
Identify clauses related to:
- Termination
- Indemnification
- Governing law
- Jurisdiction
- Liability and remedies
"""


2: Retrieve Legal Context (RAG)

In [38]:
#Initial Legal Keywords
legal_keywords = [
    "termination",
    "governing law",
    "jurisdiction",
    "liability",
    "breach"
]

#Retriever Function
def retrieve_legal_context(chunks, keywords):
    retrieved = []
    for chunk in chunks:
        if any(k.lower() in chunk.lower() for k in keywords):
            retrieved.append(chunk)
    return retrieved

3: Combine Retrieved Chunks

In [39]:
retrieved_legal_chunks = retrieve_legal_context(
    contract_chunks,
    legal_keywords
)

combined_legal_context = "\n\n".join(retrieved_legal_chunks)

print("Retrieved legal clauses:", len(retrieved_legal_chunks))


Retrieved legal clauses: 5


4: Run Legal Agent

In [40]:
def run_legal_agent(context):
    clauses = []

    if "termination" in context.lower():
        clauses.append("Termination for cause clause identified.")

    if "liability" in context.lower():
        clauses.append("Limitation of liability clause identified.")

    return {
        "extracted_clauses": clauses,
        "risk_level": "Medium",
        "confidence": 0.70
    }


In [41]:
legal_output = run_legal_agent(combined_legal_context)


5: Validate Legal Output

In [42]:
def validate_legal_output(output):
    issues = []

    if not output["extracted_clauses"]:
        issues.append("No legal clauses extracted")

    return {
        "is_valid": len(issues) == 0,
        "issues": issues
    }


In [43]:
legal_validation = validate_legal_output(legal_output)


6: Legal Risk Summary

In [44]:
legal_summary = {
    "risk_level": legal_output["risk_level"],
    "confidence": legal_output["confidence"],
    "validated": legal_validation["is_valid"]
}


7: Package Legal Pipeline Output

In [45]:
final_legal_pipeline_output = {
    "query_template": LEGAL_QUERY.strip(),
    "retrieved_chunks_count": len(retrieved_legal_chunks),
    "extracted_clauses": legal_output["extracted_clauses"],
    "validation": legal_validation,
    "summary": legal_summary
}


✅ Task 1: Add Keyword "indemnification"

Update keywords 👇

In [46]:
legal_keywords = [
    "termination",
    "governing law",
    "jurisdiction",
    "liability",
    "breach",
    "indemnification"   # ✅ added
]


✅ Task 2: Re-run Legal Retrieval

In [47]:

retrieved_legal_chunks = retrieve_legal_context(
    contract_chunks,
    legal_keywords
)

combined_legal_context = "\n\n".join(retrieved_legal_chunks)

print("Retrieved legal clauses:", len(retrieved_legal_chunks))

Retrieved legal clauses: 5


In [48]:
def run_legal_agent(context):
    clauses = []

    text = context.lower()

    if "termination" in text:
        clauses.append("Termination clause identified.")

    if "liability" in text:
        clauses.append("Limitation of liability clause identified.")

    if "indemnif" in text:   # catches indemnify / indemnification
        clauses.append("Indemnification obligation identified.")

    return {
        "extracted_clauses": clauses,
        "risk_level": "High",      # ⬆️ increased
        "confidence": 0.85         # ⬆️ increased
    }


In [49]:
legal_output = run_legal_agent(combined_legal_context)
legal_validation = validate_legal_output(legal_output)


Operations Pipeline 

The operations pipeline retrieves execution-related clauses, extracts timelines and deliverables, validates them, and produces an operational risk summary.

1: Define Operations Query Template

This guides the Operations Agent.

In [50]:
OPERATIONS_QUERY = """
Identify clauses related to:
- Service levels (SLA)
- Timelines and milestones
- Deliverables
- Uptime and availability commitments
"""


2: Retrieve Operations Context (RAG)

In [51]:
#Initial Operations Keywords 
operations_keywords = [
    "sla",
    "timeline",
    "deliverable",
    "milestone",
    "service level"
]



In [52]:

#Retriever Function
def retrieve_operations_context(chunks, keywords):
    retrieved = []
    for chunk in chunks:
        if any(k.lower() in chunk.lower() for k in keywords):
            retrieved.append(chunk)
    return retrieved

3: Combine Retrieved Chunks

In [ ]:
#“This chunk was added for demonstration.”

contract_chunks.append(
    "The service provider guarantees 99.9% uptime and shall meet all SLA requirements."
)


In [57]:
retrieved_operations_chunks = retrieve_operations_context(
    contract_chunks,
    operations_keywords
)

combined_operations_context = "\n\n".join(retrieved_operations_chunks)

print("Retrieved operations clauses:", len(retrieved_operations_chunks))


Retrieved operations clauses: 1


4: Run Operations Agent

In [58]:
def run_operations_agent(context):
    clauses = []
    text = context.lower()

    if "sla" in text or "service level" in text:
        clauses.append("Service level obligations identified.")

    if "deliverable" in text:
        clauses.append("Deliverables definition identified.")

    if "timeline" in text or "milestone" in text:
        clauses.append("Timeline or milestone commitments identified.")

    return {
        "extracted_clauses": clauses,
        "risk_level": "Medium",
        "confidence": 0.70
    }


In [59]:
operations_output = run_operations_agent(combined_operations_context)


5: Validate Operations Output

In [60]:
def validate_operations_output(output):
    issues = []

    if not output["extracted_clauses"]:
        issues.append("No operational obligations identified")

    return {
        "is_valid": len(issues) == 0,
        "issues": issues
    }

operations_validation = validate_operations_output(operations_output)

6: Operations Risk Summary


In [61]:
operations_summary = {
    "risk_level": operations_output["risk_level"],
    "confidence": operations_output["confidence"],
    "validated": operations_validation["is_valid"]
}

7: Package Operations Pipeline Output


In [62]:
final_operations_pipeline_output = {
    "query_template": OPERATIONS_QUERY.strip(),
    "retrieved_chunks_count": len(retrieved_operations_chunks),
    "extracted_clauses": operations_output["extracted_clauses"],
    "validation": operations_validation,
    "summary": operations_summary
}

Task 1: Add Keyword "uptime"

Update keywords 👇

In [63]:
operations_keywords = [
    "sla",
    "timeline",
    "deliverable",
    "milestone",
    "service level",
    "uptime"   # ✅ added
]

Task 2: Re-run Operations Retrieval


In [64]:
retrieved_operations_chunks = retrieve_operations_context(
    contract_chunks,
    operations_keywords
)

combined_operations_context = "\n\n".join(retrieved_operations_chunks)

print("Retrieved operations clauses:", len(retrieved_operations_chunks))

Retrieved operations clauses: 1


4 (Re-run Operations Agent with Uptime)

In [65]:
def run_operations_agent(context):
    clauses = []
    text = context.lower()

    if "sla" in text or "service level" in text:
        clauses.append("Service level obligations identified.")

    if "uptime" in text:
        clauses.append("Uptime availability commitment identified.")

    if "deliverable" in text:
        clauses.append("Deliverables definition identified.")

    if "timeline" in text or "milestone" in text:
        clauses.append("Timeline or milestone commitments identified.")

    return {
        "extracted_clauses": clauses,
        "risk_level": "High",     # ⬆️ increased
        "confidence": 0.85        # ⬆️ increased
    }


In [66]:
operations_output = run_operations_agent(combined_operations_context)
operations_validation = validate_operations_output(operations_output)
print(operations_output)
print(operations_validation)

{'extracted_clauses': ['Service level obligations identified.', 'Uptime availability commitment identified.'], 'risk_level': 'High', 'confidence': 0.85}
{'is_valid': True, 'issues': []}


Coordinator: Merging Agent Outputs

The coordinator aggregates domain-specific pipeline outputs, computes overall risk and confidence, and produces a unified JSON response.

1: Input: Pipeline Outputs

In [67]:
legal_output = final_legal_pipeline_output
compliance_output = final_compliance_pipeline_output
finance_output = final_finance_pipeline_output
operations_output = final_operations_pipeline_output

2: Define Final Output Schema (Conceptual)

In [68]:
{
  "domains": {
    "legal": {...},
    "compliance": {...},
    "finance": {...},
    "operations": {...}
  },
  "overall_risk": "...",
  "overall_confidence": 0.00,
  "highest_risk_domain": "...",
  "highest_risk_clauses": [...]
}


{'domains': {'legal': {Ellipsis},
  'compliance': {Ellipsis},
  'finance': {Ellipsis},
  'operations': {Ellipsis}},
 'overall_risk': '...',
 'overall_confidence': 0.0,
 'highest_risk_domain': '...',
 'highest_risk_clauses': [Ellipsis]}

3: Merge Pipeline Outputs (FIXED & CORRECT)

In [69]:
def coordinator_merge(legal, compliance, finance, operations):
    return {
        "legal": legal,
        "compliance": compliance,
        "finance": finance,
        "operations": operations
    }


In [70]:
#usage
merged_output = coordinator_merge(
    legal_output,
    compliance_output,
    finance_output,
    operations_output
)

print(merged_output.keys())


dict_keys(['legal', 'compliance', 'finance', 'operations'])


4: Compute Overall Risk

In [71]:
RISK_PRIORITY = {
    "High": 3,
    "Medium": 2,
    "Low": 1
}


In [72]:
def compute_overall_risk(merged):
    highest = ("Low", None)

    for domain, output in merged.items():
        risk = output["summary"]["risk_level"]
        if RISK_PRIORITY[risk] > RISK_PRIORITY[highest[0]]:
            highest = (risk, domain)

    return highest


In [73]:
#usage
overall_risk, highest_risk_domain = compute_overall_risk(merged_output)


5: TASK 1 - Confidence Aggregation Logic 

In [74]:
def aggregate_confidence(merged):
    confidences = [
        output["summary"]["confidence"]
        for output in merged.values()
        if output["summary"]["confidence"] is not None
    ]

    return round(sum(confidences) / len(confidences), 2)


In [75]:
#usage
overall_confidence = aggregate_confidence(merged_output)


6: TASK 2 - Print Highest-Risk Clause(s)

In [76]:
def get_highest_risk_clauses(merged, highest_domain):
    return merged[highest_domain].get("extracted_clauses", [])

In [77]:
highest_risk_clauses = get_highest_risk_clauses(
    merged_output,
    highest_risk_domain
)

print("⚠️ Highest Risk Domain:", highest_risk_domain)
print("⚠️ Highest Risk Clauses:")
for clause in highest_risk_clauses:
    print("-", clause)

⚠️ Highest Risk Domain: legal
⚠️ Highest Risk Clauses:
- Termination for cause clause identified.
- Limitation of liability clause identified.


7: Build Final JSON Output (Coordinator Output)

In [78]:
final_coordinator_output = {
    "domains": merged_output,
    "overall_risk": overall_risk,
    "highest_risk_domain": highest_risk_domain,
    "highest_risk_clauses": highest_risk_clauses,
    "overall_confidence": overall_confidence
}


In [79]:
import json
from datetime import datetime

final_coordinator_output["timestamp"] = datetime.now().isoformat()

with open("final_coordinator_output.json", "w") as f:
    json.dump(final_coordinator_output, f, indent=4)

print("✅ Final coordinator output saved")


✅ Final coordinator output saved


“The coordinator aggregates multi-domain pipeline outputs, computes overall risk using priority rules, and derives a unified confidence-weighted decision artifact.”